**Movie-Show Recommender ChatBot**

This project creates a ChatBot which can suggest viewers what movies or shows to watch based on the input it gets from user such as genre, actors, maturity rating etc.

In [ ]:
!pip install openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import pandas as pd

# Set your OpenAI API key
client = OpenAI(
    api_key="sk-proj-vuAWkL1IEVoOS6rSlNxjT3BlbkFJzjJjr72iaS0fZss0Ng2R",
)

# Function to clean and load Netflix data from CSV
def load_netflix_data(path='/content/drive/MyDrive/ChatBot/Dataset.csv'):
    netflix_df = pd.read_csv('/content/drive/MyDrive/ChatBot/Dataset.csv')
    netflix_df.fillna("", inplace=True)  # Replace NaN with empty strings
    netflix_data = []
    for _, row in netflix_df.iterrows():
        netflix_data.append({
            "title": row['title'].strip().lower(),
            "type": row['type'].strip().lower(),
            "description": row['description'].strip(),
            "rating": row['rating'].strip(),
            "listed_in": row['listed_in'].strip().lower(),
            "duration": row['duration'].strip(),
            "date_added": row['date_added'].strip(),
            "cast": row['cast'].strip().lower(),
            "director": row['director'].strip(),
            "release_year": row['release_year'],
            "country": row['country'].strip()
        })
    return netflix_data

# Load the Netflix data from CSV file
netflix_data = load_netflix_data('/content/drive/MyDrive/ChatBot/Dataset.csv')

# Function to list unique genres for debugging
def list_unique_genres():
    unique_genres = set()
    for item in netflix_data:
        genres = item['listed_in'].split(", ")
        for genre in genres:
            unique_genres.add(genre)

list_unique_genres()

def generate_response(user_input):
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful Netflix recommendation assistant."},
            {"role": "user", "content": user_input}
        ],
        model="gpt-3.5-turbo",
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.7,
    )
    message = chat_completion.choices[0].message.content.strip()
    return message


def get_recommendations(content_type=None, genre=None, actor=None):
    filtered_data = netflix_data
    if content_type:
        filtered_data = [item for item in filtered_data if content_type in item['type']]
    if genre:
        filtered_data = [item for item in filtered_data if genre in item['listed_in']]
    if actor:
        filtered_data = [item for item in filtered_data if actor in item['cast']]

    if not filtered_data:
        return "I'm sorry, I don't have any recommendations for that criteria at the moment."

    recommendations = "\n".join([
        f"{item['title']} (Rating: {item['rating']}): {item['description']} - {item['listed_in']}"
        for item in filtered_data[:5]  # Limiting to top 5 recommendations
    ])
    return f"Here are some recommendations:\n{recommendations}"

def extract_genre_and_actor(user_input_lower):
    genre = None
    actor = None
    tokens = user_input_lower.split()
    if "movie" in tokens:
        genre_index = tokens.index("movie") + 1
    elif "show" in tokens:
        genre_index = tokens.index("show") + 1
    else:
        genre_index = -1

    if genre_index > 0 and genre_index < len(tokens):
        genre = tokens[genre_index].strip()

    if "actor" in user_input_lower:
        actor = user_input_lower.split("actor")[-1].strip()

    return genre, actor

# Interactive function for user input
def get_user_input():
    user_input = input("Enter your request: ")
    user_input_lower = user_input.lower()

    genre, actor = extract_genre_and_actor(user_input_lower)
    content_type = "movie" if "movie" in user_input_lower else "show" if "show" in user_input_lower else None

    if content_type:
        recommendations = get_recommendations(content_type, genre, actor)
        print(recommendations)
    else:
        response_message = generate_response(user_input)
        print(response_message)

# Example Us
get_user_input()

Enter your request: horror movies
Here are some recommendations:
dick johnson is dead (Rating: PG-13): As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable. - documentaries
my little pony: a new generation (Rating: PG): Equestria's divided. But a bright-eyed hero believes Earth Ponies, Pegasi and Unicorns should be pals — and, hoof to heart, she’s determined to prove it. - children & family movies
sankofa (Rating: TV-MA): On a photo shoot in Ghana, an American model slips back in time, becomes enslaved on a plantation and bears witness to the agony of her ancestral past. - dramas, independent movies, international movies
the starling (Rating: PG-13): A woman adjusting to life after a loss contends with a feisty bird that's taken over her garden — and a husband who's struggling to find a way forward. - comedies, dramas
je suis karl (Rating: TV-MA): After most of her family is murdered in a